In [1]:
# paramter cell do not remove!!
nb_parm=''

In [2]:
# Parameters
nb_parm = "bridnok"


In [3]:
import sys
import os

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

## Do the task After this

In [4]:
import time

time.sleep(10)

In [5]:
libraries = [
    'pyspark', 'findspark', 'boto3', 'papermill', 'pyarrow', 'pandas', 
    'langchain', 'ollama', 'langchain_ollama', 'langchain_chroma'
]

for library in libraries:
    try:
        exec(f"import {library}")
        print(f"{library} is installed.")
    except ImportError:
        print(f"{library} is NOT installed.")


pyspark is installed.
findspark is installed.
boto3 is installed.


papermill is installed.
pyarrow is installed.
pandas is installed.
langchain is installed.


ollama is installed.


langchain_ollama is installed.


langchain_chroma is installed.


In [6]:
"""
Since Ollama and Jupyter Notebook run in separate containers, 
Jupyter needs to communicate with Ollama via HTTP requests, 
because they are not in the same process or environment.
"""

'\nSince Ollama and Jupyter Notebook run in separate containers, \nJupyter needs to communicate with Ollama via HTTP requests, \nbecause they are not in the same process or environment.\n'

In [7]:
from langchain_ollama.llms import OllamaLLM

# Explicitly define Ollama's API base URL
llm = OllamaLLM(
    model="deepseek-r1:7b",
    temperature=0,
    # base_url="http://ollama:11434"
    base_url="http://host.docker.internal:11434" # need to refer because it run as separate container the langchain not know what the end point of model are so this is need to be attached

)

# Define a simple question prompt
question = "What is the capital of thailand?"
response = llm.invoke(question)

# Print the response
print(response)


<think>

</think>

The capital of Thailand is Bangkok.


In [8]:
# import weaviate

# # Establish a connection to the Weaviate instance
# client = weaviate.connect_to_custom(
#     http_host="host.docker.internal",
#     http_port=8082,
#     http_secure=False,  # Set to True if using HTTPS
#     grpc_host="host.docker.internal",
#     grpc_port=50051,
#     grpc_secure=False,  # Set to True if using a secure gRPC connection
# )

# # Check if the client is ready
# print(client.is_ready())  # Should return True if the connection is successful

# # Define the schema for the object (if you haven't already defined it)
# # Example: Create a simple schema for 'Person' with 'name' and 'age' properties
# schema = {
#     "classes": [
#         {
#             "class": "Person",
#             "description": "A class to represent people",
#             "properties": [
#                 {
#                     "name": "name",
#                     "dataType": ["text"]
#                 },
#                 {
#                     "name": "age",
#                     "dataType": ["int"]
#                 }
#             ]
#         }
#     ]
# }

# # Check if the schema contains the 'Person' class using the correct method
# if "Person" not in client.schema.get():
#     client.schema.create(schema)

# # Now let's add a person object to the database
# person_data = {
#     "name": "John Doe",
#     "age": 30
# }

# # Write the object to Weaviate
# client.data_object.create(
#     data_object=person_data,
#     class_name="Person"
# )

# # Now let's list the objects stored in the "Person" class
# results = client.data_object.get(class_name="Person")

# # Print out the stored objects
# for obj in results['data']['Get']['Person']:
#     print(f"Name: {obj['name']}, Age: {obj['age']}")

# # Close the connection
# client.close()


In [9]:
# import bs4
# from langchain import hub
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.vectorstores import Chroma
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_community.embeddings import OllamaEmbeddings

# #### INDEXING ####

# # Load Documents
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# # Split
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# # Embed
# vectorstore = Chroma.from_documents(documents=splits, 
#                                     embedding=OllamaEmbeddings(
#                                                 model="mxbai-embed-large",  # Or another embedding model, like `nomic-embed-text`
#                                                 base_url="http://host.docker.internal:11434"
#                                             )
#                                    )

# retriever = vectorstore.as_retriever()

# #### RETRIEVAL and GENERATION ####

# # Prompt
# prompt = hub.pull("rlm/rag-prompt")

# # LLM
# llm = llm = OllamaLLM(
#     model="deepseek-r1:7b",
#     temperature=0,
#     base_url="http://host.docker.internal:11434" 
# )

# # Post-processing
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# # Chain
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# # Question
# rag_chain.invoke("What is Task Decomposition?")